<a href="https://colab.research.google.com/github/Abhijith-Nagarajan/PadhAI_Implementations/blob/main/PyTorch_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import numpy as np

In [ ]:
import time

#### Numpy vs Torch

In [ ]:
t1_cpu = torch.rand(10000,10000)
t2_cpu = torch.rand(10000,10000)
n1 = np.random.rand(10000,10000)
n2 = np.random.rand(10000,10000)

In [ ]:
%timeit
torch.matmul(t1_cpu,t2_cpu)

tensor([[2499.9170, 2493.0967, 2480.2927,  ..., 2465.6812, 2514.3569,
         2483.8689],
        [2540.5813, 2527.1001, 2526.7402,  ..., 2506.1538, 2546.0852,
         2532.7439],
        [2540.2122, 2548.6917, 2546.8420,  ..., 2508.9221, 2566.0166,
         2555.8655],
        ...,
        [2504.7231, 2474.8550, 2495.3953,  ..., 2460.7991, 2495.2456,
         2494.7983],
        [2504.6807, 2530.6089, 2494.1211,  ..., 2480.5576, 2524.7429,
         2513.0647],
        [2523.5400, 2519.4895, 2523.3132,  ..., 2509.4067, 2548.7229,
         2529.1426]])

In [ ]:
%timeit
np.matmul(n1,n2)

array([[2494.48525009, 2510.9557902 , 2501.27198668, ..., 2542.19756653,
        2529.59649212, 2517.05596801],
       [2492.04236243, 2532.09265513, 2487.90038279, ..., 2531.46015206,
        2544.00345436, 2505.05793283],
       [2460.46887452, 2451.59334611, 2453.55103164, ..., 2483.40292216,
        2486.46785132, 2457.29986758],
       ...,
       [2489.53173066, 2490.35439993, 2484.20228343, ..., 2522.37800905,
        2524.27809434, 2484.08090673],
       [2468.54952269, 2504.44917688, 2495.78776398, ..., 2529.1117802 ,
        2507.81712966, 2502.10996621],
       [2483.60196609, 2490.52045157, 2486.87966377, ..., 2510.76372723,
        2529.8636999 , 2490.42027807]])

#### Torch with GPU

In [ ]:
# Connecting to a GPU
torch.cuda.device_count()

1

In [ ]:
torch.cuda.device(0)

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
cuda_obj = torch.device('cuda:0')

In [ ]:
start = time.time()
for i in range(10):
    t1 = torch.rand([10000,10000],device=cuda_obj)
    t2 = torch.rand([10000,10000],device=cuda_obj)
    print(t1+t2)
    print(torch.matmul(t1,t2))
end = time.time()
print(end-start)

#### Feedforward Network with PyTorch

In [3]:
t1 = torch.rand(3,4)

In [ ]:
neurons_per_layer = list(map(int,[input(f'Enter neurons for layer {i}: ') for i in range(1,4)]))
neurons_per_layer = torch.FloatTensor(neurons_per_layer)

Enter neurons for layer 1: 4
Enter neurons for layer 2: 3
Enter neurons for layer 3: 2


In [ ]:
t1/torch.sqrt(neurons_per_layer[0])

tensor([[0.2591, 0.1642, 0.1772, 0.1601],
        [0.1361, 0.3000, 0.3852, 0.0795],
        [0.4592, 0.1302, 0.3072, 0.2970]])

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [ ]:
class FFNetwork_Params(nn.Module):
    def __init__(self, layers:int, neurons_per_layer:torch.tensor, X:torch.tensor, target_classes:int):
        super().__init__()
        self.no_layers = layers
        self.neurons_per_layer = neurons_per_layer
        self.weights = []
        self.bias = []
        self.A = []
        self.H = []
        self.loss_info = dict()
        rows,features = X.shape

        '''
        include in parameter
        match update_method:
            case 'mini-batch':
                self.batch_size = 128
            case 'sgd':
                self.batch_size = 1

        self.optimizer = optimizer
        '''

        for layer in range(layers+1):
            weights_sqrt = torch.sqrt(neurons_per_layer[layer])
            if layer==0:
                weights_row = (features,neurons_per_layer[layer])
            elif layer<layers:
                weights_row = (neurons_per_layer[layer-1],neurons_per_layer[layer])
            else:
                weights_row = (neurons_per_layer[layer],target_classes)

            weights_info = torch.rand(weights_row,neurons_per_layer[layer])/weights_sqrt
            self.weights.append(nn.Parameter(weights_info))
            self.bias.append(nn.Parameter(torch.rand(self.batch_size,neurons_per_layer[layer])))

        print('Feedforward network has been initialized')

    def derivative_of_activation(self,layer):
        return self.H[layer]*(1-self.H[layer])

    def sigmoid_operation(self,a):
        return 1.0/1.0+torch.exp(-a)


    def forward_pass(self,X):
        for layer in range(self.layers):
            a = torch.matmul(X,self.weights[layer])+self.bias[layer] if layer==0 else torch.matmul(self.H[layer-1],self.weights[layer])+self.bias[layer]
            self.A.append(a)
            self.H.append(self.sigmoid_operation(a))


    def fit(self,X,lr,epochs):
        for epoch in range(epochs):
            Y_pred = self.forward_pass()
